In [1]:
import requests
import pandas as pd
import numpy as np
import re
from bs4 import BeautifulSoup
import time

In [2]:
def get_page_content(request_url):
    #得到页面内容
    header={"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/86.0.4240.111 Safari/537.36"}
    html=requests.get(request_url,headers=header,timeout=10)
    content=html.text
    soup=BeautifulSoup(content,'html.parser',from_encoding='utf-8')
    return soup

In [3]:
request_url="https://ditie.mapbar.com/beijing_line/"
soup=get_page_content(request_url)
subways=soup.find_all('div',class_='station')

df=pd.DataFrame(columns=['name','site'])
for subway in subways:
    #得到线路名称
    route_name=subway.find('strong',class_='bolder').text
    #找到线路中，每一站的名称
    routes=subway.find('ul')
    routes=routes.find_all('a')
    for route in routes:
        temp={'name':route.text,'site':route_name}
        df=df.append(temp,ignore_index=True)

df['city']='北京'
df.to_excel('./subway.xlsx',index=False)

D:\ProgramData\Anaconda3\lib\site-packages\bs4\__init__.py:203: UserWarning: You provided Unicode markup but also provided a value for from_encoding. Your from_encoding will be ignored.
  warnings.warn("You provided Unicode markup but also provided a value for from_encoding. Your from_encoding will be ignored.")


In [12]:
# 添加经度longitude,维度 latitude
def get_location(keyword,city):
    header = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/86.0.4240.111 Safari/537.36"}
    
    request_url='http://restapi.amap.com/v3/place/text?key=31bb2b7831c8588465c41ab947e213ac&keywords='+\
                keyword+'&types=&city='+city+'&children=1&offset=1&page=1&extensions=all'
    data=requests.get(request_url,headers=header)
    data.encoding='utf-8'
    data=data.text
    """
     .* 具有贪婪的性质，首先匹配到不能匹配为止
    """
    pattern='location":"(.*?),(.*?)"'
    result=re.findall(pattern,data)
    try:
        return result[0][0],result[0][1]
    except:
        return get_location(keyword.replace('站',''),city)

In [ ]:
# 这里在pycharm运行过一遍，有现成数据，再这里就不再次重复运行，避免封号
df=pd.read_excel('./subway.xlsx')
df['longitude'],df['latitude']=None,None

for index,row in df.iterrows():
    longitude,latitude=get_location(row['name'],row['city'])
    df.iloc[index]['longitude']=longitude
    df.iloc[index]['latitude']=latitude
    sleep_time=np.random.randint(100,200)/1000
    time.sleep(sleep_time)
    print(index,row['name'],longitude,latitude)

df.to_excel('./subway.xlsx',index=False)

In [8]:
# 数据加载
data=pd.read_excel('./subway.xlsx')

In [9]:
#计算两点之间距离
def compute_distance(longtitue1,latitude1,longitude2,latitude2):
    header = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/86.0.4240.111 Safari/537.36"}
    request_url='http://restapi.amap.com/v3/distance?key=31bb2b7831c8588465c41ab947e213ac&origins='+\
                str(longtitue1)+","+str(latitude1)+'&destination='+str(longitude2)+','+str(latitude2)+"&type=1"
    data=requests.get(request_url,headers=header)
    data.encoding='utf-8'
    data=data.text
    print(data)
    pattern='distance":"(.*?)","duration":"(.*)"'
    result=re.findall(pattern,data)
    return result[0][0]

In [ ]:
# 同理这里的数据在pycharm运行过，不再重复操作
from collections import defaultdict
# 保存图中两点间距离
graph=defaultdict(dict)

for i in range(data.shape[0]):
    site1=data.iloc[i]['site']
    if i<data.shape[0]-1:
        site2=data.iloc[i+1]['site']
        #如果是同一条线路
        if site1==site2:
            longitude1,latitude1=data.iloc[i]['longitude'],data.iloc[i]['latitude']
            longitude2,latitude2=data.iloc[i+1]['longitude'],data.iloc[i+1]['latitude']
            name1=data.iloc[i]['name']
            name2=data.iloc[i+1]['name']
            # 按照距离，计算两点之间的距离
            distance=compute_distance(longitude1,latitude1,longitude2,latitude2)
            graph[name1][name2]=distance
            graph[name2][name1]=distance
            print(name1,name2,distance)

{"status":"1","info":"OK","infocode":"10000","results":[{"origin_id":"1","dest_id":"1","distance":"4143","duration":"480"}]}
四惠站 四惠东站 "4143
{"status":"1","info":"OK","infocode":"10000","results":[{"origin_id":"1","dest_id":"1","distance":"102118","duration":"7440"}]}
四惠东站 高碑店站 "102118
{"status":"1","info":"OK","infocode":"10000","results":[{"origin_id":"1","dest_id":"1","distance":"111744","duration":"9600"}]}
高碑店站 传媒大学站 "111744
{"status":"1","info":"OK","infocode":"10000","results":[{"origin_id":"1","dest_id":"1","distance":"4827","duration":"1080"}]}
传媒大学站 双桥站 "4827
{"status":"1","info":"OK","infocode":"10000","results":[{"origin_id":"1","dest_id":"1","distance":"3607","duration":"720"}]}
双桥站 管庄站 "3607
{"status":"1","info":"OK","infocode":"10000","results":[{"origin_id":"1","dest_id":"1","distance":"3242","duration":"540"}]}
管庄站 八里桥站 "3242
{"status":"1","info":"OK","infocode":"10000","results":[{"origin_id":"1","dest_id":"1","distance":"3409","duration":"480"}]}
八里桥站 通州北苑站 "3409
{"st

In [ ]:
# 数据保存到pickle文件
import pickle
output=open('graph.pkl','wb')
graph=pickle.dump(graph,output)

In [2]:
import pickle
file=open('graph.pkl','rb')
graph=pickle.load(file)
print(graph)

defaultdict(<class 'dict'>, {'四惠站': {'四惠东站': '4143', '大望路站': '4415'}, '四惠东站': {'四惠站': '4143', '高碑店站': '102118'}, '高碑店站': {'四惠东站': '102118', '传媒大学站': '111744'}, '传媒大学站': {'高碑店站': '111744', '双桥站': '4827'}, '双桥站': {'传媒大学站': '4827', '管庄站': '3607'}, '管庄站': {'双桥站': '3607', '八里桥站': '3242'}, '八里桥站': {'管庄站': '3242', '通州北苑站': '3409'}, '通州北苑站': {'八里桥站': '3409', '果园站': '1472'}, '果园站': {'通州北苑站': '1472', '九棵树站': '1510'}, '九棵树站': {'果园站': '1510', '梨园站': '2323'}, '梨园站': {'九棵树站': '2323', '临河里站': '2727'}, '临河里站': {'梨园站': '2727', '土桥站': '1674'}, '土桥站': {'临河里站': '1674'}, '西二旗站': {'生命科学园站': '6812', '上地站': '2862', '龙泽站': '6892'}, '生命科学园站': {'西二旗站': '6812', '朱辛庄站': '3479'}, '朱辛庄站': {'生命科学园站': '3479', '巩华城站': '6680', '育知路站': '2964'}, '巩华城站': {'朱辛庄站': '6680', '沙河站': '6498'}, '沙河站': {'巩华城站': '6498', '沙河高教园站': '3845'}, '沙河高教园站': {'沙河站': '3845', '南邵站': '6825'}, '南邵站': {'沙河高教园站': '6825', '北邵洼站': '1981'}, '北邵洼站': {'南邵站': '1981', '昌平东关站': '2741'}, '昌平东关站': {'北邵洼站': '2741', '昌平站': '9602'}, '昌平站': {'昌平东关站': '9602', '十三

In [3]:
# 找到开销最小的节点
def find_lowest_cost_node(costs):
    # 初始化数据
    lowest_cost = float('inf')  # 打擂法，初始化最小值 inf
    lowest_cost_node = None
    # 遍历所有节点
    for node in costs:
        if not node in processed:
            # 如果当前节点的开销比已经存在的开销小，那么就更新节点为开销最小的节点
            if costs[node] < lowest_cost:
                lowest_cost = costs[node]
                lowest_cost_node = node
    return lowest_cost_node

In [4]:
#找到最短路径
def find_shortest_path():
    node=end
    shortest_path=[end]
    while parents[node]!=start:
        # 往前移动一步
        node=parents[node]
        #往前移动一步，添加到路径中
        shortest_path.append(node)
    shortest_path.append(start)
    return shortest_path

In [5]:
# 计算图中从start到end的最短距离
def dijkstra():
    #查询到目前开销最小的节点(从U种)
    node=find_lowest_cost_node(costs)
    #print('当前cost最小节点:',node)
    #使用找到的开销最小节点，计算它的邻居，是否可以通过它进行更新
    #如果所有节点，都在processed，就结束
    while node is not None:
        #获取节点的cost
        cost=costs[node] # cost 是node到start
        #获取节点的邻居
        neighbors=graph[node]
        #遍历所有邻居，看是否可以通过它进行更新
        for neighbor in neighbors.keys():
            #计算邻居到当前节点+当前节点开销
            new_cost=cost+float(neighbors[neighbor])
            if neighbor not in costs or new_cost<costs[neighbor]:
                costs[neighbor]=new_cost
                #经过node达到neighbor节点，cost更少
                parents[neighbor]=node
        #将当前节点，标记为已处理
        processed.append(node)
        #下一步，继续找U中最短距离的节点 costs=U,processed=S
        node=find_lowest_cost_node(costs)
        #print('当前状态最小节点:',node)
    #循环完，说明所有节点处理完毕
    startest_path=find_shortest_path()
    startest_path.reverse()
    return startest_path

In [20]:
def compute(site1,site2):
    global start,end,costs,parents,processed
    start = site1
    end = site2
    # 创建节点的开销表,cost是指从start到该节点的距离
    costs = {}
    # 存储父节点的Hash表，用于记录路径
    parents = {}
    parents[end] = None

    # 获取节点相邻的节点
    for node in graph[start].keys():
        costs[node] = float(graph[start][node])
        parents[node] = start
    # 终点到起点，设置无穷大
    costs[end] = float('inf')
    processed = []
    shortest_path=dijkstra()
    return shortest_path

In [7]:
site1='五道口站'
site2='北京南站'
shortest_path=compute(site1,site2)
print(shortest_path)

['五道口站', '知春路站', '大钟寺站', '西直门站', '新街口站', '平安里站', '西四站', '灵境胡同站', '西单站', '宣武门站', '菜市口站', '陶然亭站', '北京南站']


In [8]:
def get_nearest_subway(data,longitude1,latitude1):
    #保存当前最小距离
    distance=float('inf')
    longitude1,latitude1=longitude1,latitude1
    nearst=None
    for i in range(data.shape[0]):
        site1=data.iloc[i]['name']
        longitude=float(data.iloc[i]['longitude'])
        latitude=float(data.iloc[i]['latitude'])
        temp=(float(longitude1)-longitude)**2+(float(latitude1)-latitude)**2
        if temp<distance:
            distance=temp
            nearest=site1
    return nearest

In [25]:
def compute2(site1,site2):
    longitude1,latitude1=get_location(site1,city)
    longitude2,latitude2=get_location(site2,city)
    # 加载所有的地铁线路
    data=pd.read_excel('./subway.xlsx')
    #求site1,site2最近的地铁站
    start=get_nearest_subway(data,longitude1,latitude1)
    end=get_nearest_subway(data,longitude2,latitude2)
    #print('{}最近的地铁站为{}'.format(site1,start))
    #print('{}最近的地铁站为{}'.format(site2,end))
    shortest_path=compute(start, end)
    if site1!=start:
        shortest_path.insert(0,site1)
    if site2!=end:
        shortest_path.append(site2)
    print('从{}=》{}的最优路径为:\n {}'.format(site1,site2,shortest_path))

In [26]:
city='北京'
site1='清华大学'
site2='798'
compute2(site1,site2)

从清华大学=》798的最优路径为:
 ['清华大学', '五道口站', '知春路站', '西土城站', '牡丹园站', '健德门站', '北土城站', '安贞门站', '惠新西街南口站', '芍药居站', '望京西站', '望京站', '阜通站', '望京南站', '将台站', '798']
